In [1]:
import sqlite3
import numpy as np
import pandas as pd
import seaborn as sns
import numpy as np
import datetime
import sys
import os

# PROCESSING WILDFIRE DATASET

In [2]:
def create_wildfire_df(PATH):
    '''
    PATH: Linux path to sqlite for wildfire database
    Output:
        dataframe: of wildfire with the following columns:
        LATITUDE
        LONGITUDE
        STATE
        date
        fire_size
        fire_year
        fire_month
        fire_total (This is total fire in a particular latitude, longitude, fire_year and fire_month)
    '''
    assert(os.path.exists(PATH)),"Incorrect file path"
    conn = sqlite3.connect(PATH)
    data_dates = pd.read_sql_query("SELECT fire_year, discovery_date FROM fires;", conn)
    data_size = pd.read_sql_query("SELECT fire_size, fire_size_class FROM fires;", conn)
    data_location = pd.read_sql_query("SELECT latitude, longitude, state FROM fires;", conn)
    date = pd.read_sql_query("select datetime(DISCOVERY_DATE) as DISCOVERY_DATE from fires;", conn)
    data_dates_arr = date['DISCOVERY_DATE']
    fire_year_arr = data_dates['FIRE_YEAR']
    fire_size_arr = data_size['FIRE_SIZE']  
    #Create a dataframe df 
    df = data_location
    df['date'] = data_dates_arr
    df['fire_size'] = fire_size_arr
    df['fire_year'] = fire_year_arr
    df['fire_month'] = pd.DatetimeIndex(df['date']).month
    #Converted Lat and Long to int to remove the exact precision
    df['LATITUDE'] = df['LATITUDE'].astype(int)
    df['LONGITUDE'] = df['LONGITUDE'].astype(int)
##Compute total fire based on Latitude, Longitude, fire_year and fire_month
    df['fire_total'] = df.groupby(['LATITUDE','LONGITUDE', 'fire_year','fire_month'])['fire_size'].transform(sum)
    
    return df

In [3]:
def large_fire_coord(state, fire_size, df):
    '''
    Input: 
        state: The state we want to focus on.
        df: Input dataframe
        fire_size: fire size above which is considered large
    Output:
        lat_min,lat_max,long_min,long_max: the coordinates with fires above fire_size in state 'state'
    '''
    #print(state)
    df = df.query("fire_total >= fire_size and STATE == @state")
    lat_max = df['LATITUDE'].max()
    lat_min = df['LATITUDE'].min()
    long_max = df['LONGITUDE'].max()
    long_min = df['LONGITUDE'].min() 
    return lat_min,lat_max,long_min,long_max

In [11]:
def process_df(coords, df):
    '''
    Input: 
        coords: A tuple of lat_min,lat_max,long_min,long_max in which we need to find wild fires
        Find sum of fire occuring within the location, for a given year and month
    
    '''
    lat_min,lat_max,long_min,long_max = coords
    #print(lat_min,lat_max,long_min,long_max)
    df_new = df.query("LATITUDE >= @lat_min and LATITUDE <= @lat_max and LONGITUDE >= @long_min and LONGITUDE <= @long_max")
    #print(df_new)
    df_new['fire_total_area'] = df_new.groupby(['fire_year','fire_month'])['fire_total'].transform(sum)
    df_new = df_new.sort_values('fire_total', ascending=False).drop_duplicates(['fire_year','fire_month'])
    df_final = df_new[['fire_year','fire_month','fire_total_area']]
    return df_final

In [4]:
PATH = '../../188-million-us-wildfires/FPA_FOD_20170508.sqlite'

In [5]:
fire_df = create_wildfire_df(PATH)

In [8]:
coords = large_fire_coord('AK',10000,fire_df)

In [12]:
fire_df_local = process_df(coords,fire_df)

C:\Users\payal\Anaconda4\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


In [13]:
fire_df_local

,fire_year,fire_month,fire_total_area
211296,1997,6,3.600862e+06
305585,2004,6,1.229897e+07
1215267,2009,6,3.240731e+06
305684,2004,7,2.813613e+06
1796561,2015,6,2.494216e+07
305353,2002,7,2.367513e+06
305816,2005,6,7.291983e+06
305317,2002,6,1.769609e+06
306311,2007,7,9.018486e+05
305976,2005,7,3.522544e+06


# Working on temperature dataset

In [25]:
import os 
import numpy as np
import iris   
import iris.plot as iplt
import iris.quickplot as qplt
import matplotlib.pyplot as plt
%matplotlib inline
import iris.coord_categorisation as cat
import pandas as pd
from iris.analysis import Aggregator

In [76]:
def region_based_cube(cube,coords):
    '''
    Inputs:
        cube with latitude and longitude as coords
        coords: the region we want to extract from input cube
    Outputs: 
        A smaller cube within coords
    '''
    lat_min,lat_max,long_min,long_max = coords
    lat_cons = iris.Constraint(latitude = lambda x : lat_min < x < lat_max)
    if (long_min<0):
        long_min = long_min+360
    if(long_max <0) :
        long_max = long_max+360
    long_cons = iris.Constraint(longitude = lambda x : long_min < x < long_max)
    new_cube = cube.extract(lat_cons & long_cons)
    return new_cube

In [20]:
##Adding auxilary axis:
def get_decade(coord, value):
    date = coord.units.num2date(value)
    return date.year - date.year % 10
def get_year(coord, value):
    date = coord.units.num2date(value)
    return date.year

def get_month(coord, value):
    date = coord.units.num2date(value)
    return date.month


#cat.add_categorised_coord(cube, 'decade', 'time', get_decade)

In [21]:
def get_monthy_mean(cube):
    '''
    Output: Returns an array which is a monthy mean tempurature over the entire area of the cube for all the years.
    '''
    cube_mean = cube.collapsed(['latitude','longitude'], iris.analysis.MEAN)
    cube_mean_month = cube_mean.aggregated_by(['month'],iris.analysis.MEAN)
    return cube_mean_month.data

In [53]:
def compute_diff(data,axis,month):
    if (len(data) < 12):
        return 0
    new_data = data[month] - monthy_mean.data[month]
    return new_data

In [44]:
PATH_temp = os.path.join(os.getcwd(), "../../air.mon.mean.nc")

In [45]:
cube_temp = iris.load_cube(PATH_temp)
##Add year and month as auxillary axis to cube
cat.add_categorised_coord(cube_temp, 'year', 'time', get_year)
cat.add_categorised_coord(cube_temp, 'month', 'time', get_month)


In [78]:
##Extract region based cube
cube_local = region_based_cube(cube_temp,coords)

##Compute monthy mean in the local cube for the entire duration. This returns an array of shape (12,).
monthy_mean = get_monthy_mean(cube_local)

C:\Users\payal\Anaconda4\lib\site-packages\iris\cube.py:3218: UserWarning: Collapsing spatial coordinate 'latitude' without weighting
  warnings.warn(msg.format(coord.name()))
C:\Users\payal\Anaconda4\lib\site-packages\iris\coords.py:1410: UserWarning: Collapsing a non-contiguous coordinate. Metadata may not be fully descriptive for 'latitude'.
  warnings.warn(msg.format(self.name()))
C:\Users\payal\Anaconda4\lib\site-packages\iris\coords.py:1410: UserWarning: Collapsing a non-contiguous coordinate. Metadata may not be fully descriptive for 'longitude'.
  warnings.warn(msg.format(self.name()))


In [36]:
DIFF = Aggregator('diff_mean',
                         compute_diff,
                         units_func=lambda units: 1)


In [79]:
##Iterate over months and find difference and put in one array. 
##This gives how much above/below average is the temperature of a month/year in the cube
cube_mean = cube_local.collapsed(['latitude','longitude'], iris.analysis.MEAN)
for i in range(12):
    cube_AK_max_year = cube_mean.aggregated_by(['year'],DIFF,month=i)
    print(cube_AK_max_year.shape)
    if i == 0 :
        new_arr = cube_AK_max_year.data.filled()
    else:
        new_arr = np.vstack((new_arr,cube_AK_max_year.data.filled()))

C:\Users\payal\Anaconda4\lib\site-packages\iris\cube.py:3218: UserWarning: Collapsing spatial coordinate 'latitude' without weighting
  warnings.warn(msg.format(coord.name()))
C:\Users\payal\Anaconda4\lib\site-packages\iris\coords.py:1410: UserWarning: Collapsing a non-contiguous coordinate. Metadata may not be fully descriptive for 'latitude'.
  warnings.warn(msg.format(self.name()))
C:\Users\payal\Anaconda4\lib\site-packages\iris\coords.py:1410: UserWarning: Collapsing a non-contiguous coordinate. Metadata may not be fully descriptive for 'longitude'.
  warnings.warn(msg.format(self.name()))


(73,)
(73,)
(73,)
(73,)
(73,)
(73,)
(73,)
(73,)
(73,)
(73,)
(73,)
(73,)


In [81]:
##Coherant to wildfire timelines
years = [i+1948 for i in range(73)]
months = [i for i in range(1,13)]

In [82]:
temp_pd = pd.DataFrame(new_arr,columns=cols)
small_years = [1992+i for i in range(24)]
temp_pd = temp_pd[small_years]
temp_pd['month'] = [i for i in range(1,13)]

In [83]:
##Rearrange pd: with columns as months, years and values as: "how much above/below average the temp is."
temp_pd_mod = temp_pd.melt(id_vars=['month'], var_name='year', value_name='temp_diff')

In [84]:
##Use wildfire pd: 
fire_df_local = fire_df_local.rename(columns={"fire_year": "year", "fire_month": "month"})

In [85]:
##Merge only the intersection of two pandas
result1 = pd.merge(temp_pd_mod, fire_df_local, on=['year', 'month'])

,month,year,temp_diff,fire_total_area
0,4,1992,-0.306427,31.40
1,5,1992,-1.476074,22655.10
2,6,1992,0.647583,160720.40
3,7,1992,0.341827,97579.10
4,8,1992,-0.328735,9703.40
5,9,1992,-4.393036,50.90
6,10,1992,-2.912598,4.10
7,4,1993,2.805573,436.10
8,5,1993,1.593170,12313.50
9,6,1993,0.755035,405537.90


In [96]:
max_temp  = result1['temp_diff'].max()

In [106]:
result1.query('fire_total_area>20000').sort_values('fire_total_area')

,month,year,temp_diff,fire_total_area
171,9,2012,0.242493,2.201590e+04
134,6,2008,-0.419159,2.231140e+04
1,5,1992,-1.476074,2.265510e+04
93,5,2003,0.170166,2.278430e+04
59,5,1999,-0.742615,2.310190e+04
24,5,1995,2.084961,2.402500e+04
181,8,2013,1.169586,2.773340e+04
110,4,2005,0.508240,2.895210e+04
26,7,1995,0.083466,3.397510e+04
53,7,1998,0.841980,3.670720e+04


In [98]:
result1.query("temp_diff == @max_temp")

,month,year,temp_diff,fire_total_area
89,11,2002,6.013245,5.0
